# Projeto 2 - Ciência dos Dados

## **Membros**

**Gabriel Salvator Benatar**

**Matheus Freitas Sant'Ana**

In [58]:
%matplotlib inline
import pandas as pd

import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [59]:
df = pd.read_csv('HRR_Scorecard_20.csv')

In [68]:
df.head()

,HRR,Total Hospital Beds,Total ICU Beds,Available Hospital Beds,Potentially Available Hospital Beds*,Available ICU Beds,Potentially Available ICU Beds*,Adult Population,Population 65+,Projected Infected Individuals,...,"Percentage of Potentially Available ICU Beds Needed, Six Months","Percentage of Total ICU Beds Needed, Six Months","ICU Beds Needed, Twelve Months","Percentage of Available ICU Beds Needed, Twelve Months","Percentage of Potentially Available ICU Beds Needed, Twelve Months","Percentage of Total ICU Beds Needed, Twelve Months","ICU Beds Needed, Eighteen Months","Percentage of Available ICU Beds Needed, Eighteen Months","Percentage of Potentially Available ICU Beds Needed, Eighteen Months","Percentage of Total ICU Beds Needed, Eighteen Months"
2,"Akron, OH","1,358",186,518,938,94.0,140,"547,990","111,042","109,598",...,236%,178%,165,176%,118%,89%,108,115%,77%,58%
3,"Alameda County, CA","2,695",293,665,"1,680",139.0,216,"1,310,189","214,991","262,038",...,352%,259%,380,273%,176%,130%,248,178%,115%,85%
4,"Albany, GA",704,60,221,462,27.0,43,"157,143","30,466","31,429",...,219%,157%,47,174%,109%,78%,31,115%,72%,52%
5,"Albany, NY","4,804",425,"1,579","3,191",193.0,309,"1,477,723","318,695","295,545",...,292%,212%,452,234%,146%,106%,294,152%,95%,69%
6,"Albuquerque, NM","2,908",380,"1,102","2,005",161.0,270,"1,336,795","279,457","267,359",...,301%,214%,406,252%,150%,107%,265,165%,98%,70%
